In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
import numpy as np
import  torch, os
import  numpy as np
from    subgraph_data_processing import Subgraphs,Subgraphs_syn
import  pickle
import numpy as np

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--gpu_id', type=int, default=0, help='gpu id')
parser.add_argument('--dataset', type=str, default='cora')
parser.add_argument('--dis_metric', type=str, default='ours')
parser.add_argument('--epochs', type=int, default=2000)
parser.add_argument('--nlayers', type=int, default=3)
parser.add_argument('--hidden', type=int, default=256)
parser.add_argument('--lr_adj', type=float, default=0.01)
parser.add_argument('--lr_feat', type=float, default=0.01)
parser.add_argument('--lr_model', type=float, default=0.01)
parser.add_argument('--weight_decay', type=float, default=0.0)
parser.add_argument('--dropout', type=float, default=0.0)
parser.add_argument('--normalize_features', type=bool, default=True)
parser.add_argument('--keep_ratio', type=float, default=1.0)
parser.add_argument('--reduction_rate', type=float, default=0.0025)
parser.add_argument('--seed', type=int, default=15, help='Random seed.')
parser.add_argument('--alpha', type=float, default=0, help='regularization term.')
parser.add_argument('--debug', type=int, default=0)
parser.add_argument('--sgc', type=int, default=1)
parser.add_argument('--inner', type=int, default=0)
parser.add_argument('--outer', type=int, default=20)
parser.add_argument('--save', type=int, default=0)
parser.add_argument('--one_step', type=int, default=0)

parser.add_argument('--epoch', type=int, help='epoch number', default=10)
parser.add_argument('--n_way', type=int, help='n way', default=3)
# parser.add_argument('--k_spt', type=int, help='k shot for support set', default=3)
# parser.add_argument('--k_qry', type=int, help='k shot for query set', default=24)
parser.add_argument('--k_spt', type=int, help='k shot for support set', default=1)
parser.add_argument('--k_qry', type=int, help='k shot for query set', default=3)
parser.add_argument('--task_num', type=int, help='meta batch size, namely task num', default=32)

parser.add_argument('--meta_lr', type=float, help='meta-level outer learning rate', default=1e-4) # 之前是1e-3，为了处理合成数据集上的梯度消失问题，改成1e-4

parser.add_argument('--update_lr', type=float, help='task-level inner update learning rate', default=1e-2)
parser.add_argument('--update_step', type=int, help='task-level inner update steps', default=10)
parser.add_argument('--update_step_test', type=int, help='update steps for finetunning', default=20)
parser.add_argument('--input_dim', type=int, help='input feature dim', default=1)
parser.add_argument('--hidden_dim', type=int, help='hidden dim', default=256)
parser.add_argument('--attention_size', type=int, help='dim of attention_size', default=32)
parser.add_argument("--data_dir", default="D:\pythonProject\python_file\Graph_DD\G-Meta-master\arxiv", type=str, required=False, help="The input data dir.")
parser.add_argument("--no_finetune", default=True, type=str, required=False, help="no finetune mode.")
parser.add_argument("--task_setup", default='Disjoint', type=str, required=False, help="Select from Disjoint or Shared Setup. For Disjoint-Label, single/multiple graphs are both considered.")
parser.add_argument("--method", default='G-Meta', type=str, required=False, help="Use G-Meta")
parser.add_argument('--task_n', type=int, help='task number', default=1)
parser.add_argument("--task_mode", default='False', type=str, required=False, help="For Evaluating on Tasks")
parser.add_argument("--val_result_report_steps", default=100, type=int, required=False, help="validation report")
parser.add_argument("--train_result_report_steps", default=200, type=int, required=False, help="training report")
parser.add_argument("--num_workers", default=0, type=int, required=False, help="num of workers")
parser.add_argument("--batchsz", default=10000, type=int, required=False, help="batch size")
parser.add_argument("--link_pred_mode", default='False', type=str, required=False, help="For Link Prediction")
parser.add_argument("--h", default=2, type=int, required=False, help="neighborhood size")
parser.add_argument('--sample_nodes', type=int, help='sample nodes if above this number of nodes', default=300)
args = parser.parse_args([])

In [3]:
device='cuda:1'

In [4]:
root = r"../DATA/arxiv/"
feat = np.load( '../DATA/arxiv/features.npy', allow_pickle = True)
with open('../DATA/arxiv/graph_dgl.pkl', 'rb') as f:
    dgl_graph = pickle.load(f)
with open('../DATA/arxiv/label.pkl', 'rb') as f:
    info = pickle.load(f)
total_class = len(np.unique(np.array(list(info.values()))))
print('There are {} classes '.format(total_class))

feat = [feat]  
feat_shape = feat[0].shape[1]


dgl_graph = [dgl_graph]


db_train = Subgraphs(root, 'train', info, n_way=args.n_way, k_shot=args.k_spt, k_query=args.k_qry, batchsz=args.batchsz, args = args, adjs = dgl_graph, h = args.h)
db_val = Subgraphs(root, 'val', info, n_way=args.n_way, k_shot=args.k_spt,k_query=args.k_qry, batchsz=100, args = args, adjs = dgl_graph, h = args.h)
db_test = Subgraphs(root, 'test', info, n_way=args.n_way, k_shot=args.k_spt,k_query=args.k_qry, batchsz=100, args = args, adjs = dgl_graph, h = args.h)
print('------ Start Training ------')

There are 40 classes 
shuffle DB :train, b:10000, 3-way, 1-shot, 3-query, 2-hops
shuffle DB :val, b:100, 3-way, 1-shot, 3-query, 2-hops
shuffle DB :test, b:100, 3-way, 1-shot, 3-query, 2-hops
------ Start Training ------


In [5]:
# from Meta_learner import Meta_learner
from  subgraph_data_processing import Subgraphs_syn
import torch
from torch.utils.data import DataLoader
import numpy as np
import dgl
from utils import *
import deeprobust.graph.utils as utils
import torch.nn as nn
from utils import match_loss, regularization, row_normalize_tensor
from models.parametrized_adj import PGE
from meta import Meta
import copy

def collate(samples):
    graphs_spt, labels_spt, graph_qry, labels_qry, center_spt, center_qry, nodeidx_spt, nodeidx_qry, support_graph_idx, query_graph_idx = map(list, zip(*samples))

    return graphs_spt, labels_spt, graph_qry, labels_qry, center_spt, center_qry, nodeidx_spt, nodeidx_qry, support_graph_idx, query_graph_idx

class Meta_learner():
    # 初始化函数，接收数据、参数和设备信息
    def __init__(self, db_train,db_val,db_test, args, device, feat_shape= None,**kwargs):
        self.db_train = db_train
        self.db_val = db_val
        self.db_test = db_test
        self.args = args
        self.device = device
        
        path = r"../DATA/arxiv/train.csv"
        
        ratio = 0.0025 # 0.25%
        n,labels_syn= self.generate_labels_from_csv(path,ratio)
        self.config = [('GraphConv', [feat_shape, self.args.hidden_dim])]
        
        if self.args.h > 1:
            self.config  = self.config  + [('GraphConv', [self.args.hidden_dim, self.args.hidden_dim])] * (self.args.h - 1)

        self.config  = self.config  + [('Linear', [self.args.hidden_dim, args.n_way])]
        self.maml = Meta(args, self.config,self.device).to(device)
        # 计算需要生成的synthetic graph的节点数
        n = int(n)        
        # 初始化synthetic graph的节点数和特征
        # 获取特征维度
        d = 128
        self.feat_syn = nn.Parameter(torch.FloatTensor(n, d).to(device))

        # 初始化PGE模型
        self.pge = PGE(nfeat=d, nnodes=n, device=self.device, args=self.args).to(device)
        
        # 生成synthetic graph的标签
        self.labels_syn = torch.LongTensor(labels_syn).to(device)

        self.dictLabels = self.loadCSV(path)
        # 初始化特征优化器和PGE模型优化器
        self.optimizer_feat = torch.optim.Adam([self.feat_syn], lr=args.lr_feat)
        self.optimizer_pge = torch.optim.Adam(self.pge.parameters(), lr=args.lr_adj)
        
        # 打印synthetic graph的节点数和特征维度
        print('adj_syn:', (n,n), 'feat_syn:', self.feat_syn.shape)

    def reset_parameters(self):
        self.feat_syn.data.copy_(torch.randn(self.feat_syn.size()))

    def loadCSV(self,path):
        import csv
        from collections import Counter
        dictLabels = {}

        # dictLabels（标签到子图的映射）、

        with open(path, 'r') as csvfile:
            csvreader = csv.reader(csvfile, delimiter=',')
            next(csvreader, None)  # skip (filename, label)
            for i, row in enumerate(csvreader):
                filename = row[1]
                g_idx = int(filename.split('_')[0])
                label = row[2]
                # append filename to current label

                if label in dictLabels.keys():
                    dictLabels[label].append(filename)
                else:
                    dictLabels[label] = [filename]

        return dictLabels

    def generate_labels_from_csv(self,file_path,ratio):
        import csv
        from collections import Counter
        with open(file_path, 'r') as csv_file:
            reader = csv.DictReader(csv_file)
            data = list(reader)

        counter = Counter()  # 使用 Counter 对标签进行计数
        num_class_dict = {}  # 存储每个类别需要生成的节点数
        n = len(data)  # 数据总数

        # 统计每个类别的出现次数
        for row in data:
            label = int(row['label'])
            counter[label] += 1

        # 根据标签出现次数进行排序
        sorted_counter = sorted(counter.items(), key=lambda x: x[1])
        sum_ = 0
        labels_syn = []  # 合成标签列表
        syn_class_indices = {}  # 记录每个类别的索引范围
    #     ratio = 0.08
        # 计算每个类别需要生成的节点数，并生成合成标签
        for ix, (c, num) in enumerate(sorted_counter):

            if ix == len(sorted_counter) - 1:
                num_class_dict[c] = int(n * ratio) - sum_  
                # 如果是最后一个类别，那么直接计算需要生成的节点数，并将这个类别的合成标签添加到 labels_syn 列表中
                syn_class_indices[c] = [len(labels_syn), len(labels_syn) + num_class_dict[c]]
                labels_syn += [c] * num_class_dict[c]

            else:
                num_class_dict[c] = max(int(num * ratio), 1)  # 其他类别的节点数
                # 如果不是最后一个类别，那么先计算需要生成的节点数，然后将这个类别的合成标签添加到 labels_syn 列表中。
                sum_ += num_class_dict[c]
                syn_class_indices[c] = [len(labels_syn), len(labels_syn) + num_class_dict[c]]
                labels_syn += [c] * num_class_dict[c]

        return n * ratio,labels_syn

    def get_sub_adj_feat(self,features, labels_syn, label_dict):
        idx_selected = []

        from collections import Counter
        counter = Counter(labels_syn.cpu().numpy())
        selected_features = []
        for c in counter.keys():
            class_label = str(c)
            if class_label in label_dict:
                node_indices = [label_dict[class_label].index(node) for node in label_dict[class_label]]
                num_features = counter[c]
                selected_indices = np.random.choice(node_indices, size=num_features, replace=False)

                idx_selected.extend(selected_indices)

                selected_features.extend(features[selected_indices])

        selected_features = np.array(selected_features)

        return selected_features
    
    def generate_syn_feat_db(self,dictLabels,feat):

#         dictLabels = loadCSV(path)
        self.feat_syn = torch.from_numpy(self.get_sub_adj_feat(feat, self.labels_syn, dictLabels)).to(self.device)
    # # 重置synthetic graph的特征
    # reset_parameters()
        adj_syn_orj = self.pge(self.feat_syn)
        adj_syn_norm = utils.normalize_adj_tensor(adj_syn_orj, sparse=False)

        adj_syn = adj_syn_norm
        adj_syn[adj_syn < 0.5] = 0
        adj_syn[adj_syn >= 0.5] = 1

        adj_syn_norm_sp = sp.csr_matrix(adj_syn.cpu().detach().numpy())
        g = dgl.from_scipy(adj_syn_norm_sp) # 生成的合成图太过密集，导致二跳子图约等于全图
        # 将labels_syn转换为dictLabels形式的字典
        dict_labels = {f"0_{i}": self.labels_syn[i].item() for i in range(len(self.labels_syn))}
        db_syn = Subgraphs_syn('syn', dict_labels, n_way=self.args.n_way, k_shot=self.args.k_spt,k_query=self.args.k_qry, batchsz=100, args = self.args, adjs = g, h = self.args.h, labels = self.labels_syn)
       

        db = DataLoader(db_syn, self.args.task_num, shuffle=True, num_workers=self.args.num_workers, pin_memory=True, collate_fn = collate)
        
        return db,adj_syn_orj
    
    def train(self, verbose=True,feat = None,dgl_graph = None):
        # 获取参数和数据
        args = self.args
        # data = self.data
        device = self.device
        db_train = self.db_train
        db_val = self.db_val
        db_test = self.db_test
        max_acc = 0
        
        for out_loop in range(5):
            self.maml = Meta(args, self.config,device).to(device)
            model_parameters = list(self.maml.parameters())
            # 重置synthetic graph的特征
            self.reset_parameters()

            for epoch in range(self. args.epoch):
                db = DataLoader(db_train, args.task_num, shuffle=True, num_workers=args.num_workers, pin_memory=True, collate_fn = collate)
                db_syn,adj_syn  = self.generate_syn_feat_db(self.dictLabels,feat[0])
                db_iter = iter(db)  # Create an iterator for the DataLoader object
                db_syn_iter = iter(db_syn) 
                iters = min(len(db),len(db_syn))
                for step in range(iters):
                    # Extract one batch of data from db
                    x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry = next(db_iter)

                    nodes_len = 0              
                    nodes_len += sum([sum([len(j) for j in i]) for i in n_spt])
                    losses_opt,loss_q = self.maml.grad_opt(x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry, feat)

                    x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry = next(db_syn_iter)
                    nodes_len = 0             
                    nodes_len += sum([sum([len(j) for j in i]) for i in n_spt])
                    losses_opt_syn,loss_q_syn = self.maml.grad_opt_syn(x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry, feat)

                    print("gather gradients of parameters done")
                    # 计算梯度匹配损失
                    len_opt = len(losses_opt)
                    len_syn = len(losses_opt_syn)
                    len_gw = min(len_syn,len_opt )
                    # # 对于更长的一个，从后面开始截取
                    # if len_opt > len_syn:
                    #     losses_opt = losses_opt[len_opt - len_gw:]
                        
                    # elif len_syn > len_opt:
                    #     losses_opt_syn = losses_opt_syn[len_syn - len_gw:]
                        
                    # print('len_opt:',len_opt)
                    # print('len_syn:',len_syn)
                    
                    for i in range(len_gw):
                        loss = torch.tensor(0.0,requires_grad=True).to(self.device)
                       
                        gw_real = torch.autograd.grad(losses_opt[i], model_parameters,retain_graph=True)
                        
                        gw_syn  = torch.autograd.grad(losses_opt_syn[i], model_parameters, create_graph=True,retain_graph=True)

                        if i == 0:
                            print(gw_syn)
                            old_syn = gw_syn
                            old_real = gw_real
                        else:
                            print("zy521",all(torch.allclose(old, new) for old, new in zip(gw_real, old_real)))# 假设有两个梯度列表 gw_real1 和 gw_real2

                            print("ztl521",all(torch.allclose(old, new) for old, new in zip(gw_syn, old_syn)))
                            
                        gw_real = list((_.detach().clone() for _ in gw_real))
                        gw_syn  = list((_.detach().clone() for _ in gw_syn))

                        loss += match_loss(gw_syn, gw_real, args, device=self.device)
                            
                        # 计算正则化损失
                        # TODO: regularize
                        if args.alpha > 0:
                            loss_reg = args.alpha * regularization(adj_syn, utils.tensor2onehot(self.labels_syn))
                        # else:
                        else:
                            loss_reg = torch.tensor(0)

                        loss = loss + loss_reg

                        # 更新合成图
                        self.optimizer_feat.zero_grad()
                        # 每次计算梯度时，梯度都会被累加到梯度缓存中。
                        # 因此，在每次更新模型参数之前需要将梯度缓存清零，以避免梯度累加的影响。
                        self.optimizer_pge.zero_grad()
                        loss.backward() # 计算损失函数对于模型参数的梯度

                        # 根据 it 的值选择更新 self.optimizer_pge 或 self.optimizer_feat
                        if i % 2 == 0:
                            self.optimizer_pge.step() # 使用优化算法来更新模型参数
                        else:
                            self.optimizer_feat.step()

                        if i % 5 ==0: # 打印梯度匹配损失
                            print('Gradient matching loss:', loss.item())

                    loss = torch.tensor(0.0,requires_grad=True).to(self.device)
                    gw_real = torch.autograd.grad(loss_q, model_parameters)
                    gw_real = list((_.detach().clone() for _ in gw_real))
                    gw_syn  = torch.autograd.grad(loss_q_syn, model_parameters,create_graph=True)
                    gw_syn  = list((_.detach().clone() for _ in gw_syn))
                    loss += match_loss(gw_syn, gw_real, args, device=self.device)

                    # 计算正则化损失
                    # TODO: regularize
                    if args.alpha > 0:
                        loss_reg = args.alpha * regularization(adj_syn, utils.tensor2onehot(self.labels_syn))
                    # else:
                    else:
                        loss_reg = torch.tensor(0)

                    loss = loss + loss_reg

                    # 更新合成图
                    self.optimizer_feat.zero_grad()
                    self.optimizer_pge.zero_grad()
                    loss.backward() # 计算损失函数对于模型参数的梯度

                    self.optimizer_pge.step() # 使用优化算法来更新模型参数
                    self.optimizer_feat.step()
                    print('Gradient matching loss:', loss.item())
                
                    # 进行内循环，更新 GNN 模型的参数
                    self.maml.meta_optim.zero_grad()
                    loss_q_syn.backward()
                    # # 注册回调函数保存梯度
                    # self.net.parameters().register_hook(save_gradients)
                    self.maml.meta_optim.step() # update gnn param

                if epoch % 2 == 0:
                    # validation per epoch
                    db_v = DataLoader(db_val, 1, shuffle=True, num_workers=args.num_workers, pin_memory=True, collate_fn = collate)
                    accs_all_test = []

                    for x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry in db_v:

                        accs = self.maml.finetunning(x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry, feat)
                        accs_all_test.append(accs)

                    accs = np.array(accs_all_test).mean(axis=0).astype(np.float16)
                    print('Epoch:', epoch + 1, ' Val acc:', str(accs[-1])[:5])
                # if accs[-1] > max_acc:
                #     max_acc = accs[-1]
                #     model_max = copy.deepcopy(self.maml)
    
        db_t = DataLoader(db_test, 1, shuffle=True, num_workers=args.num_workers, pin_memory=True, collate_fn = collate)
        accs_all_test = []

        for x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry in db_t:
            accs = self.maml.finetunning(x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry, feat)
            accs_all_test.append(accs)

        accs = np.array(accs_all_test).mean(axis=0).astype(np.float16)
        print('Test acc:', str(accs[1])[:5])


In [6]:
Meta_learner = Meta_learner(db_train, db_val, db_test, args, device,feat_shape)

adj_syn: (251, 251) feat_syn: torch.Size([251, 128])


In [7]:
torch.backends.cudnn.enable =True
torch.backends.cudnn.benchmark = True
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [8]:
Meta_learner.train(verbose=True,feat = feat,dgl_graph = dgl_graph)


gather gradients of parameters done
(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:1',
       grad_fn=<AddBackward0>), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 

KeyboardInterrupt: 